In [1]:
import os
import numpy as np
import pandas as pd
from datetime import date

from sklearn.model_selection import KFold, train_test_split, StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss, roc_auc_score, auc, roc_curve
from sklearn.preprocessing import MinMaxScaler


In [2]:
"""
讀csv資料
"""
data_path = './Data/'
df_train = pd.read_csv(data_path+'train_offline.csv')
df_test = pd.read_csv(data_path+'test_offline.csv')

print(df_train.shape)
print(df_test.shape)

(1160742, 7)
(594142, 6)


In [3]:
#test 沒有date 欄位
df_train.columns
df_test.columns
'''

'''
#ids = ['User_id', 'Coupon_id','Merchant_id']
#df = df_train.drop(['User_id', 'Coupon_id','Merchant_id'] , axis=1)
df = df_train
df.head()

dtype_df = df.dtypes.reset_index() 
dtype_df

,index,0
0,User_id,int64
1,Merchant_id,int64
2,Coupon_id,float64
3,Discount_rate,object
4,Distance,float64
5,Date_received,float64
6,Date,float64


In [4]:

#-----------------------------------------------------------------------------------------------
def convertRate(row):
    """Convert discount to rate"""
    if ':' in row:
        rows = row.split(':')
        return 1.0 - float(rows[1])/float(rows[0])
    else:
        return float(row)
    
def getDiscountType(row):
    if row == 'null':
        return 'null'
    elif ':' in row:
        return 1
    else:
        return 0
    
def getDiscountMan(row):
    if ':' in row:
        rows = row.split(':')
        return int(rows[0])
    else:
        return 0

def getDiscountJian(row):
    if ':' in row:
        rows = row.split(':')
        return int(rows[1])
    else:
        return 0
    
def getWeekday(row):
    row = str(row)
    if(row == 'nan'):
        return np.nan
    else:
        return date(int(row[0:4]), int(row[4:6]), int(row[6:8])).weekday() + 1

def getreceived(row):
    if(row == 'nan'):
        return 0
    else:
        return 1
def processData(df):
    
    # convert distance
    df.loc[df.Distance.isna(), "Distance"] = 99
    df.loc[df.Discount_rate.isna(), "Discount_rate"] = 1.0
    
    # convert discunt_rate
    df['float_discount_rate'] = df['Discount_rate'].astype('str').apply(convertRate)
    df['discount_man'] = df['Discount_rate'].astype('str').apply(getDiscountMan)
    df['discount_jian'] = df['Discount_rate'].astype('str').apply(getDiscountJian)
    df['discount_type'] = df['Discount_rate'].astype('str').apply(getDiscountType)
    
    discount_mean = df.groupby(['User_id'])['float_discount_rate'].mean().reset_index()
    discount_mean.rename(columns={'float_discount_rate':'discount_mean'},inplace=True)
    df = pd.merge(df,discount_mean, how='left', on=['User_id'])

    #平均距離( (x * 500 公尺), 0 表示低於 500 公尺, 10 表示大於 5 公里。)
    Distance_mean = df.groupby(['Coupon_id'])['Distance'].mean().reset_index()
    Distance_mean.rename(columns={'Distance':'Distance_mean'},inplace=True)
    df = pd.merge(df,Distance_mean, how='left', on=['Coupon_id'])
    df['Distance_mean'] = df['Distance_mean']
    df['Distance_mean'] = df['Distance_mean'].fillna(99)
    
     #距離前兩大 0跟10
    df['Distance_type'] = df['Distance_mean'].apply(lambda x : 1 if x in[0,10] else 0 )
    #最大距離
    Distance_max = df.groupby(['Coupon_id'])['Distance'].max().reset_index()
    Distance_max.rename(columns={'Distance':'Distance_max'},inplace=True)
    df = pd.merge(df,Distance_max, how='left', on=['Coupon_id'])
    df['Distance_max'] = df['Distance_max'].fillna(99)
    #計次_使用者_取折價日期
    count_df = df.groupby(['User_id'])['Date_received'].agg({'Date_received_Count':'size'}).reset_index()
    df = pd.merge(df, count_df, on=['User_id'], how='left')
    df['Date_received_Count'] = df['Date_received_Count'].fillna(0)
    df['Date_received_Count_type'] = df['Date_received_Count'].apply(lambda x : 1 if x < 3 else 0 )
    df['get_Date_received'] = df['Date_received'].astype('str').apply(getreceived)

    
    count_df = df.groupby(['Coupon_id'])['Date_received'].agg({'Date_received_Count2':'size'}).reset_index()
    df = pd.merge(df, count_df, on=['Coupon_id'], how='left')
    
    
    
    count_df = df.groupby(['Merchant_id'])['Coupon_id'].agg({'Coupon_id_Count':'size'}).reset_index()
    df = pd.merge(df, count_df, on=['Merchant_id'], how='left')
    df['Coupon_id_Count'] = df['Coupon_id_Count'].fillna(0)
    df['Coupon_id_Count_type'] = df['Coupon_id_Count'].apply(lambda x : 1 if x <2 else 0 )
    
    # weekday_type :  周5和周日为1，其他为0
    df['weekday'] = df['Date_received'].astype(str).apply(getWeekday)
    df['weekday_type'] = df['weekday'].apply(lambda x : 1 if x in [5,7] else 0 )
    
    df['MID_CID'] = (df['Merchant_id'] + df['Coupon_id'])/2
    

    return df

df = processData(df)
df_test = processData(df_test)


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:76: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:83: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:88: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version


In [5]:
'''
def testP(df):
    
    return df

df = testP(df)
df_test = testP(df_test)
'''

'\ndef testP(df):\n    \n    return df\n\ndf = testP(df)\ndf_test = testP(df_test)\n'

In [6]:

def label(row):
    if np.isnan(row['Date_received']):
        return -1
    if not np.isnan(row['Date']):
        td = pd.to_datetime(row['Date'], format='%Y%m%d') -  pd.to_datetime(row['Date_received'], format='%Y%m%d')
        if td <= pd.Timedelta(15, 'D'):
            return 1
    return 0

df["label"] = df.apply(label, axis=1)

In [7]:
df.head()
#df.head()
#df.groupby(['label'])['Coupon_id_Count'].value_counts() #7610

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,float_discount_rate,discount_man,discount_jian,...,Date_received_Count,Date_received_Count_type,get_Date_received,Date_received_Count2,Coupon_id_Count,Coupon_id_Count_type,weekday,weekday_type,MID_CID,label
0,1439408,2632,NaN,1,0.0,NaN,20160217.0,1.00,0,0,...,3,0,0,NaN,45,0,NaN,0,NaN,-1
1,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN,0.95,20,1,...,3,0,1,24.0,45,0,3.0,0,5611.5,0
2,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN,0.95,20,1,...,3,0,1,8.0,45,0,6.0,0,1855.0,0
3,1832624,3381,7610.0,200:20,0.0,20160429.0,NaN,0.90,200,20,...,1,1,1,46676.0,137054,0,5.0,1,5495.5,0
4,2029232,3381,11951.0,200:20,1.0,20160129.0,NaN,0.90,200,20,...,1,1,1,26035.0,137054,0,5.0,1,7666.0,0


In [20]:
#train_Y.value_counts()
df.groupby(['label'])['Merchant_id'].value_counts() #7610
#df['Merchant_id'].value_counts()

label  Merchant_id
-1     2934           21884
       5341           19656
       3381           14273
       3532           13940
       6485           11774
       1469            9287
       760             7711
       6901            7524
       2436            7084
       450             6992
       4142            6744
       7555            6019
       2099            4785
       1433            4583
       5591            4010
       1379            3957
       1941            3897
       2970            3807
       7717            3762
       3284            3664
       3710            3401
       3621            3130
       7974            3080
       8181            2916
       1945            2801
       1169            2662
       3786            2555
       1125            2453
       1080            2361
       7113            2325
                      ...  
 1     8508               1
       8534               1
       8547               1
       8563               1
 

In [9]:
def split_train_valid(row, date_cut="20160416"):
    is_train = True if pd.to_datetime(row, format="%Y%m%d") < pd.to_datetime(date_cut, format="%Y%m%d") else False
    return is_train

df_x =df[df['label'] != -1].copy()
df_x["is_train"] = df_x["Date_received"].apply(split_train_valid)
train = df_x[df_x["is_train"]]
valid = df_x[~df_x["is_train"]]
train.reset_index(drop=True, inplace=True)
valid.reset_index(drop=True, inplace=True)
print("Train size: {}, #positive: {}".format(len(train), train["label"].sum()))
print("Valid size: {}, #positive: {}".format(len(valid), valid["label"].sum()))

Train size: 667753, #positive: 32472
Valid size: 79216, #positive: 3832


In [10]:
feature = ['Distance','float_discount_rate','discount_man','discount_jian','discount_type','Distance_mean','Distance_max','Date_received_Count','weekday','weekday_type','Date_received_Count_type','Distance_type','Date_received_Count2','Coupon_id_Count','Coupon_id_Count_type','MID_CID']

In [11]:
predictors = feature
print(predictors)

def check_model(data, predictors):
    
    classifier = lambda: SGDClassifier(
        loss='log', 
        penalty='elasticnet', 
        fit_intercept=True, 
        max_iter=100, 
        shuffle=True, 
        n_jobs=1,
        class_weight=None)

    model = Pipeline(steps=[
        ('ss', StandardScaler()),
        ('en', classifier())
    ])

    parameters = {
        'en__alpha': [ 0.001, 0.01, 0.1],
        'en__l1_ratio': [ 0.001, 0.01, 0.1]
    }

    folder = StratifiedKFold(n_splits=3, shuffle=True)
    
    grid_search = GridSearchCV(
        model, 
        parameters, 
        cv=folder, 
        n_jobs=-1, 
        verbose=1)
    grid_search = grid_search.fit(data[predictors], 
                                  data['label'])
    
    return grid_search

['Distance', 'float_discount_rate', 'discount_man', 'discount_jian', 'discount_type', 'Distance_mean', 'Distance_max', 'Date_received_Count', 'weekday', 'weekday_type', 'Date_received_Count_type', 'Distance_type', 'Date_received_Count2', 'Coupon_id_Count', 'Coupon_id_Count_type', 'MID_CID']


In [12]:
model = check_model(train, predictors)

Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed:  3.8min finished
C:\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Anaconda3\lib\site-packages\sklearn\base.py:467: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
C:\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


In [13]:
y_valid_pred = model.predict_proba(valid[predictors])
valid1 = valid.copy()
valid1['pred_prob'] = y_valid_pred[:, 1]

C:\Anaconda3\lib\site-packages\sklearn\pipeline.py:381: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


In [14]:
from sklearn.metrics import roc_auc_score, accuracy_score
auc_score = roc_auc_score(y_true=valid.label, y_score=y_valid_pred[:,1])
acc = accuracy_score(y_true=valid.label, y_pred=y_valid_pred.argmax(axis=1))
print("Validation AUC: {:.4f}, Accuracy: {:.3f}".format(auc_score, acc))

Validation AUC: 0.8341, Accuracy: 0.952


In [15]:
targetset = df_test.copy()
print(targetset.shape)
targetset = targetset[~targetset.Coupon_id.isna()]
targetset.reset_index(drop=True, inplace=True)
testset = targetset[predictors].copy()

y_test_pred = model.predict_proba(testset[predictors])
test1 = testset.copy()
test1['pred_prob'] = y_test_pred[:, 1]
print(test1.shape)

(594142, 23)
(306313, 17)


C:\Anaconda3\lib\site-packages\sklearn\pipeline.py:381: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


In [16]:
output = pd.concat((targetset[["User_id", "Coupon_id", "Date_received"]], test1["pred_prob"]), axis=1)
print(output.shape)

output.loc[:, "User_id"] = output["User_id"].apply(lambda x:str(int(x)))
output.loc[:, "Coupon_id"] = output["Coupon_id"].apply(lambda x:str(int(x)))
output.loc[:, "Date_received"] = output["Date_received"].apply(lambda x:str(int(x)))
output["uid"] = output[["User_id", "Coupon_id", "Date_received"]].apply(lambda x: '_'.join(x.values), axis=1)
output.reset_index(drop=True, inplace=True)


(306313, 4)


In [17]:
out = output.groupby("uid", as_index=False).mean()
out = out[["uid", "pred_prob"]]
out.columns = ["uid", "label"]
out.to_csv('submit.csv', index=False)
out.head()

,uid,label
0,1000020_2705_20160519,0.049696
1,1000020_8192_20160513,0.048621
2,1000065_1455_20160527,0.061314
3,1000085_8067_20160513,0.046685
4,1000086_2418_20160613,0.032748
